In [1]:
import pandas as pd

data = pd.read_csv("datasets/amazon_reviews.csv")

In [2]:
data.head()

,class_index,review_title,review_text
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...


In [3]:
data['class_index'].value_counts()

class_index
2    200000
1    200000
Name: count, dtype: int64

In [4]:
data.isnull().sum()

class_index      0
review_title    24
review_text      0
dtype: int64

In [5]:
data.fillna('', inplace=True)
data.isnull().sum()

class_index     0
review_title    0
review_text     0
dtype: int64

In [6]:
data['label'] = data['class_index'] - 1
data.head()


,class_index,review_title,review_text,label
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...,1
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...,1
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...,0
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...,1
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...,1


In [7]:
data['review'] = data['review_title'] + ' ' + data['review_text']
data.head()

,class_index,review_title,review_text,label,review
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...,1,Great CD My lovely Pat has one of the GREAT vo...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...,1,One of the best game music soundtracks - for a...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...,0,Batteries died within a year ... I bought this...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...,1,"works fine, but Maha Energy is better Check ou..."
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...,1,Great for the non-audiophile Reviewed quite a ...


In [8]:
downsized = data.sample(n=5000, random_state=42)

In [9]:
#Code to clean review text data
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
from langdetect import detect

# Download necessary NLTK data
nltk.download("stopwords")
nltk.download("wordnet")

# Initialize NLP tools
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def cleanReview(text):
    if not isinstance(text, str):
        return ""

    # 1. Convert to lowercase
    text = text.lower()

    # 2. Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # 3. Remove square brackets
    text = re.sub(r'\[.*?\]', '', text)

    # 4. Remove punctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)

    # 5. Remove numbers
    text = re.sub(r'\w*\d\w*', '', text)

    # 6. Remove special quote marks & newlines
    text = re.sub(r'[‘’“”…]', '', text)
    text = text.replace("\n", " ")

    # 7. Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])

    # 8. Spell Correction
    text = str(TextBlob(text).correct())

    # 9. Stemming
    text = " ".join([stemmer.stem(word) for word in text.split()])

    # 10. Lemmatization
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sean.lai/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sean.lai/nltk_data...


In [10]:
downsized["text"] = downsized['review'].apply(cleanReview)
downsized.head()


,class_index,review_title,review_text,label,review,text
23218,2,This is a great book,I must preface this by saying that I am not re...,1,This is a great book I must preface this by sa...,great book must prefac say religi love book re...
20731,1,Huge Disappointment.,"As a big time, long term Trevanian fan, I was ...",0,"Huge Disappointment. As a big time, long term ...",huge disappoint big time long term trevanian f...
39555,2,Wayne is tight but cant hang with Turk.,This album is hot as it wants to be. However C...,1,Wayne is tight but cant hang with Turk. This a...,wayn tight can hang turk album hot want howev ...
147506,2,Excellent,I read this book when I was in elementary scho...,1,Excellent I read this book when I was in eleme...,excel read book elementari school probabl four...
314215,1,Not about Anusara,Although this book is touted on several Anusar...,0,Not about Anusara Although this book is touted...,anusara although book rout sever anusara web s...


In [11]:
from datasets import Dataset

dataset = Dataset.from_dict(downsized)

dataset = dataset.train_test_split(test_size=0.2)

/Users/sean.lai/XcodeProjects/sentiment-analysis/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import torch
import optuna
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize_function, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Model Initialization
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

# Hyperparameter tuning function
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 2e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 5)

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=dataset["train"].shuffle(seed=42).select(range(2000)),
        eval_dataset=dataset["test"].select(range(500)),
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_loss"]  # Minimize loss

# Run hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# Print best hyperparameters
print("Best hyperparameters:", study.best_params)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 4682.13 examples/s]
[I 2025-03-03 11:26:08,591] A new study created in memory with name: no-name-48a342f7-07a4-4443-91ea-dddb88012084
Error while downloading from https://cdn-lfs.hf.co/bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1740974285&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDk3NDI4NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9iZXJ0LWJhc2UtdW5jYXNlZC82OGQ0NWUyMzRlYjRhOTI4MDc0ZGZkODY4Y2VhZDAyMTlhYjg1MzU0Y2M1M2QyMGU3NzI3NTNjNmJiOTE2OWQzP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=cJZwXOaMyLIru-5%7ENBAw6GWEJvJ64JVffu4zev5huyKOdiU1ktr3%7EHED54TQYNr96Wp7Jucy2oHbT35PQ418oQeEk6c3Z4fqmQpebch89hGxdNrgs4h0UcRzrIKmVX3p-MMAUsRNBqFxdiMd-EQys2PF-1NwzTxDXPnWh6eKZPEz4yC91O%7EvbWM2cV1-0gSiQMm1sz2q%7Eub6OtLAiz0WObBqqnvH

{'loss': 0.3809, 'grad_norm': nan, 'learning_rate': 2.7033185377869044e-05, 'epoch': 0.16}


 11%|█         | 20/189 [02:08<15:32,  5.52s/it]

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 2.5522951557876362e-05, 'epoch': 0.32}


 16%|█▌        | 30/189 [03:03<14:57,  5.64s/it]

...

{'eval_loss': 0.7470428347587585, 'eval_runtime': 18.3927, 'eval_samples_per_second': 27.185, 'eval_steps_per_second': 0.87, 'epoch': 5.0}


100%|██████████| 315/315 [43:17<00:00,  8.24s/it]


{'train_runtime': 2597.0696, 'train_samples_per_second': 3.85, 'train_steps_per_second': 0.121, 'train_loss': 0.781224983457535, 'epoch': 5.0}


100%|██████████| 16/16 [00:16<00:00,  1.06s/it]
[I 2025-03-03 18:57:01,460] Trial 9 finished with value: 0.7470428347587585 and parameters: {'learning_rate': 2.2187740204098135e-05, 'batch_size': 32, 'num_train_epochs': 5}. Best is trial 2 with value: 0.43583643436431885.


Best hyperparameters: {'learning_rate': 3.404285964026493e-05, 'batch_size': 16, 'num_train_epochs': 5}


In [14]:
# Train final model with best hyperparameters
best_params = study.best_params
final_training_args = TrainingArguments(
    output_dir="./final_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    num_train_epochs=best_params["num_train_epochs"],
    learning_rate=best_params["learning_rate"],
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model_init=model_init,
    args=final_training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()
eval_results = trainer.evaluate()
print("Final Model Evaluation:", eval_results)

/Users/sean.lai/XcodeProjects/sentiment-analysis/.venv/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 10/1250 [00:43<1:28:29,  4.28s/it]

{'loss': 0.6964, 'grad_norm': 2.757385492324829, 'learning_rate': 3.3770516763142806e-05, 'epoch': 0.04}


  2%|▏         | 20/1250 [01:31<1:41:18,  4.94s/it]

{'loss': 0.6776, 'grad_norm': 2.589836359024048, 'learning_rate': 3.3498173886020685e-05, 'epoch': 0.08}


  2%|▏         | 30/1250 [02:20<1:37:52,  4.81s/it]

...

{'eval_loss': 0.7033149600028992, 'eval_runtime': 36.8889, 'eval_samples_per_second': 27.108, 'eval_steps_per_second': 1.708, 'epoch': 5.0}


100%|██████████| 1250/1250 [2:26:33<00:00,  7.03s/it]


{'train_runtime': 8793.1634, 'train_samples_per_second': 2.274, 'train_steps_per_second': 0.142, 'train_loss': 0.23480558126568793, 'epoch': 5.0}


100%|██████████| 63/63 [00:36<00:00,  1.74it/s]

Final Model Evaluation: {'eval_loss': 0.41062840819358826, 'eval_runtime': 37.0187, 'eval_samples_per_second': 27.013, 'eval_steps_per_second': 1.702, 'epoch': 5.0}
